In [1]:
import matplotlib.pyplot as plt, numpy as np, numpy.random as npr, csv, matplotlib.pyplot as plt, pandas as pd
import torch.nn as nn, timeit, torch, pystan as ps
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import LocallyLinearEmbedding
from tqdm import trange
%matplotlib inline
from time import time
from pylab import plot, show, legend

In [2]:
sm = ps.StanModel(file="/home/postdoc/dsen/Desktop/BNN_dimension_reduction/stan_files/nn_refpoints_constrainedtheta.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cf61588642e0539b092dedc55e934141 NOW.


In [3]:
data = pd.read_excel('Folds5x2_pp.xlsx').values
X = data.transpose()
Nobs = np.shape(X)[1]
print("Nobs =", Nobs, ", dim =", np.shape(X)[0])

Nobs = 9568 , dim = 5


In [12]:
n_ref = 1000
lat_dim = 3
obs_dim = np.shape(X)[0] 
ref_points = npr.choice(Nobs,n_ref,replace=False)
embedding = LocallyLinearEmbedding(n_components=lat_dim, n_neighbors=10)
X_embedded = embedding.fit_transform(X.transpose()).transpose()

In [13]:
train = np.setdiff1d(np.arange(Nobs), ref_points)
X_ref = X[:,ref_points]
Z_ref = X_embedded[:,ref_points]
X_train = X[:,train]

In [14]:
n_in = np.shape(X_embedded)[0]
n_h = 200
n_out = np.shape(X)[0]

model = nn.Sequential(nn.Linear(n_in, n_h),
                      nn.Tanh(),
                      nn.Linear(n_h, n_out))

In [15]:
def init_normal(m):
    if type(m) == nn.Linear: nn.init.normal_(m.weight)
model.apply(init_normal)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)

In [16]:
n_iter = 1_000_000
Lss = np.zeros(n_iter)
start = time()
for epoch in range(n_iter):
    X_pred = model(torch.from_numpy(Z_ref.transpose().astype(np.float32)))
    loss = criterion(X_pred, torch.from_numpy(X_ref.transpose().astype(np.float32)))
    Lss[epoch] = loss.data
    if (epoch+1)%(n_iter/10) == 0 :
        print("Epoch: %d; loss: %.3f; time: %.1f mins" % (epoch+1, loss.item(), (time()-start)/60))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 100000; loss: 68.417; time: 6.2 mins
Epoch: 200000; loss: 39.562; time: 12.3 mins
Epoch: 300000; loss: 36.696; time: 18.6 mins
Epoch: 400000; loss: 36.287; time: 24.6 mins
Epoch: 500000; loss: 35.969; time: 31.1 mins
Epoch: 600000; loss: 35.728; time: 37.1 mins
Epoch: 700000; loss: 34.772; time: 43.4 mins
Epoch: 800000; loss: 34.339; time: 49.9 mins
Epoch: 900000; loss: 33.923; time: 56.2 mins
Epoch: 1000000; loss: 33.571; time: 62.2 mins
